# Focused Ion Beam Molecular Dynamics (fibmd) Tool

In [1]:
from IPython.display import display, clear_output, Javascript
import ipywidgets as widgets
import numpy as np
from string import Template
import hublib.use
import py3Dmol
from IPython.display import FileLink, FileLinks
from hublib.ui import Submit
#from hublib.cmd import runCommand

<IPython.core.display.Javascript object>

In [2]:
%use openmpi-1.6.3-gnu-4.7.2

# Creating widgets for input

In [3]:
#title = widgets.Label(value="FIBMD Tool", width=10)

box_layout = widgets.Layout(display='flex',
                            justify_content='center')
text_layout = widgets.Layout(width = '150px')
text_layout1 = widgets.Layout(width = '50px')
text_layout2 = widgets.Layout(width ='80px',
                              position = 'right')

#ion energy input
keVin = widgets.BoundedFloatText(
    value=2000,
    min=500,
    max=60000,
    step=100,
    layout=text_layout2,
    tooltip = 'Beam energy, in keV'
)
#x dimension for simulation box
xdimin = widgets.BoundedFloatText(
    value=5,
    min=4,
    max=40,
    step=1,
    layout=text_layout1
)
#y dimension for simulation box
ydimin = widgets.BoundedFloatText(
    value=5,
    min=4,
    max=40,
    step=1,
    layout=text_layout1
)
#z dimension for simulation box
zdimin = widgets.BoundedFloatText(
    value=2,
    min=1,
    max=40,
    step=1,
    layout=text_layout1
)
# number of processors to use in MPI
pin = widgets.BoundedIntText(
    value = 2,
    min = 1,
    max = 2,
    step = 1,
    layout=text_layout2
)
#temperature setting for the silicon target
tempin = widgets.BoundedFloatText(
    value = 300.0,
    step = 10.0,
    layout = text_layout2,
    min = 10.0,
    max = 2000.
)
#mumber of gallium ions to fire
nin = widgets.BoundedIntText(
    value = 1,
    min = 1,
    max = 100,
    step = 1,
    layout=text_layout2
)
#output frequency of full .xyz file, in timesteps
ain  = widgets.BoundedIntText(
    value = 500,
    min = 0,
    max = 1e6,
    step = 100,
    layout=text_layout2
)
#time between ion impacts
dtin = widgets.BoundedFloatText(
    value = 1,
    step = 0.1,
    layout=text_layout2,
    min = 0.1
)
#angle of incidence, from z axis
phizin = widgets.BoundedFloatText(
    value = 0.0,
    step = 1,
    min = 0,
    max = 89,
    layout=text_layout2
)
#angle of incidence, from x axis
phixyin = widgets.FloatText(
    value = 0.0,
    step = 1,
    layout=text_layout2
)
#FWHM of incident beam distribution, in nm
fin = widgets.BoundedFloatText(
    value = 2,
    layout=text_layout2,
    min = 0.1
)

dimlabel = widgets.Label(" Target dimensions, nm",layout=box_layout)

xlabel = widgets.Label("    X:", position='right')
ylabel = widgets.Label("    Y:", position='right')
zlabel = widgets.Label("    Z:", position='right')
xbox = widgets.HBox([xlabel,xdimin],layout = text_layout)
ybox = widgets.HBox([ylabel,ydimin],layout = text_layout)
zbox = widgets.HBox([zlabel,zdimin],layout = text_layout)

keVlabel = widgets.Label("Ion Energy (eV):")
proclabel = widgets.Label("# of Processors:")
templabel = widgets.Label("Temperature (K):")
ionlabel = widgets.Label("# of Ions to fire:")
outlabel = widgets.Label("Output frequency:")
dtlabel = widgets.Label("$\\Delta t _{ion}$ (ps)")
anglezlabel = widgets.Label("Beam Angle $\\theta$:")
anglexylabel = widgets.Label("Beam Angle $\Phi$:")
fwhmlabel = widgets.Label("FWHM of beam (nm):")
beamlabel = widgets.Label("Effective Beam Current (nA): ")
sizelabel = widgets.Label("Estimated Total File Size (MB)")
fluxlabel = widgets.Label("$4\sigma$ Surface Flux (ions/cm$^2$ps): ")

dimin = widgets.HBox([xbox,ybox,zbox])
vlabels = widgets.VBox([keVlabel, proclabel, templabel, ionlabel, outlabel, dtlabel, anglezlabel, anglexylabel, fwhmlabel])
vinput = widgets.VBox([keVin, pin, tempin, nin, ain, dtin, phizin, phixyin, fin])

inputs2 = widgets.HBox([vlabels,vinput])

#silicon lattice constant
silat = 0.5431

## dynamically updating boxes for current, estimated file size, flux
BeamCurrent = widgets.Text(
    value = '%0.3f'%(1.602e-19/(dtin.value*1e-12)*1e9),
    layout = text_layout2,
    position = 'right'
)

SurfFlux = widgets.Text(
    value = '%1.4E'%(1/dtin.value/((fin.value/2.355*4*1e-7)**2*np.pi/4)
    ),
    layout = text_layout2,
        position = 'right'
)

    #6.44e-5 is estimate of MB per atom per snapshot
    #floors of dimin are unit cells, 8 is atoms per unit cell
    #.1/.02e-3 is estimating number of timesteps in fast regime (lasting .1 ps, with .02e-3 step size)
    #(dtin.value-.1)/.2e-3 is # of timesteps during the rest of the time
    #ain.value/10 and ain.value normalize total timesteps by output frequency in each case
    #whole process happens nin.value times
EstimatedSize = widgets.Text( 
    value = '%0.3f'%(6.5e-05*(np.floor(xdimin.value/silat) * 
                      np.floor(ydimin.value/silat) * 
                      np.floor(zdimin.value/silat))*8 *
             (.1/.02e-3 /(ain.value/10) + (dtin.value-.1)/.2e-3/(ain.value))*nin.value
),
        layout = text_layout2,
        position = 'right'
)

vlabels2 = widgets.VBox([beamlabel,fluxlabel,sizelabel])
outlabels = widgets.VBox([BeamCurrent,SurfFlux,EstimatedSize])
outlabels2 = widgets.HBox([vlabels2,outlabels])
input1 = widgets.VBox([dimlabel,dimin,inputs2],layout=widgets.Layout(border='solid'))
FullBox = widgets.VBox([input1,outlabels2])

In [4]:
#more detailed descriptions of input for tab view

desc2_layout = widgets.Layout(overflow_x='scroll',
                    width='200px',
                    height='',
                    flex_flow='row')#,
#                    display='flex')
desc1_layout = widgets.Layout(width = '80px')

headerdesc1 = widgets.Label(
    value = '',
    layout = desc1_layout
)
dimdesc1 = widgets.Label(
    value = 'Dimensions:',
    layout = desc1_layout
)
energydesc1 = widgets.Label(
    value = 'Ion Energy:',
    layout = desc1_layout
)
procdesc1 = widgets.Label(
    value = 'Processors:',
    layout = desc1_layout
)
tempdesc1 = widgets.Label(
    value = 'Temperature:',
    layout = desc1_layout
)
ionsdesc1 = widgets.Label(
    value = 'Ions:',
    layout = desc1_layout
)
outdesc1 = widgets.Label(
    value = 'Output:',
    layout = desc1_layout
)
timedesc1 = widgets.Label(
    value = '$\\Delta t _{ion}$:',
    layout = desc1_layout
)
phizdesc1 = widgets.Label(
    value = '$\\theta$:',
    layout = desc1_layout
)
phixdesc1 = widgets.Label(
    value = '$\Phi$:',
    layout = desc1_layout
)
fwhmdesc1 = widgets.Label(
    value = 'FWHM:',
    layout = desc1_layout
)

currdesc1 = widgets.Label(
    value = 'Current:',
    layout = desc1_layout
)
fluxdesc1 = widgets.Label(
    value = 'Flux:',
    layout = desc1_layout
)
sizedesc1 = widgets.Label(
    value = 'Size:',
    layout = desc1_layout
)

headerdesc2 = widgets.Label(
    value = '$ \\text{Descriptions (scrolling)} $',
    layout = desc2_layout
)
dimdesc2 = widgets.Label(
    value = '$ \\text{Simulation domain dimensions in nm. For simulation, this input is rounded to the nearest integer unit cell value.} $',
    layout = desc2_layout
)
energydesc2 = widgets.Label(
    value = '$ \\text{Kinetic Energy of the Ga ion being fired, in eV}$',
    layout = desc2_layout
)
procdesc2 = widgets.Label(
    value = '$ \\text{The number of processors to use. Limited to 2 for local nanohub simulations. Using 2 should always be preferable, but very small domains may require 1.}$',
    layout = desc2_layout
)
tempdesc2 = widgets.Label(
    value = '$ \\text{Silicon thermostatted temperature, applied to a 1.5 nm thick outer boundary in the x-y plane. Implemented as a Berendsen thermostat with } \\tau \\text{ = 0.1 (ps)} $',
    layout = desc2_layout
)
ionsdesc2 = widgets.Label(
    value = '$ \\text{Total number of ions fired at the surface.} $',
    layout = desc2_layout
)
outdesc2 = widgets.Label(
    value = '$ \\text{Snapshot every }x \\text{ timesteps during large steps (0.2 fs), } x/10 \\text{ steps during small steps (0.02 fs)} $',
    layout = desc2_layout
)
timedesc2 = widgets.Label(
    value = '$ \\text{Time between ion impacts, in ps.} $',
    layout = desc2_layout
)
phizdesc2 = widgets.Label(
    value = '$ \\text{Off-normal angle of incidence for ion beam.} $',
    layout = desc2_layout
)
phixdesc2 = widgets.Label(
    value = '$ \\text{Projected x-y plane beam direction from x axis, for } \\theta \\neq 0.$',
    layout = desc2_layout
)
fwhmdesc2 = widgets.Label(
    value = '$\\text{Full width half maximum of Gaussian ion distribution.}$',
    layout = desc2_layout
)

currdesc2 = widgets.Label(
    value = '$\\text{Beam current conversion, based on time between impacts.}$',
    layout = desc2_layout
)
fluxdesc2 = widgets.Label(
    value = '$\\text{Beam flux conversion, based on time between impacts and FWHM. This is over an area corresponding to 4$\\sigma$ of the random ion distribution.} $',
    layout = desc2_layout
)
sizedesc2 = widgets.Label(
    value = '$\\text{Estimated final .xyz file size, will tend to overestimate very small/low energy cases, may underestimate large and high energy cases.} $',
    layout = desc2_layout
)

headerdesc = widgets.HBox([headerdesc1,headerdesc2])
dimdesc = widgets.HBox([dimdesc1,dimdesc2])
energydesc = widgets.HBox([energydesc1,energydesc2])
procdesc = widgets.HBox([procdesc1,procdesc2])
tempdesc = widgets.HBox([tempdesc1,tempdesc2])
ionsdesc = widgets.HBox([ionsdesc1,ionsdesc2])
outdesc = widgets.HBox([outdesc1,outdesc2])
timedesc = widgets.HBox([timedesc1,timedesc2])
phizdesc = widgets.HBox([phizdesc1,phizdesc2])
phixdesc = widgets.HBox([phixdesc1,phixdesc2])
fwhmdesc = widgets.HBox([fwhmdesc1,fwhmdesc2])
currdesc = widgets.HBox([currdesc1,currdesc2])
fluxdesc = widgets.HBox([fluxdesc1,fluxdesc2])
sizedesc = widgets.HBox([sizedesc1,sizedesc2])

#descriptionsl = widgets.VBox([headerdesc1,dimdesc1,tempdesc1,ionsdesc1,outdesc1,timedesc1,phizdesc1,phixdesc1,fwhmdesc1,currdesc1,fluxdesc1,sizedesc1])
#descriptionsr = widgets.VBox([headerdesc2,dimdesc2,tempdesc2,ionsdesc2,outdesc2,timedesc2,phizdesc2,phixdesc2,fwhmdesc2,currdesc2,fluxdesc2,sizedesc2])
#descriptions = widgets.VBox([descriptionsl,descriptionsr])
descriptions1 = widgets.VBox([headerdesc,dimdesc,energydesc,procdesc,tempdesc,ionsdesc,outdesc,timedesc,phizdesc,phixdesc,fwhmdesc],layout=widgets.Layout(border='solid'))
descriptions2 = widgets.VBox([currdesc,fluxdesc,sizedesc])
descriptions = widgets.VBox([descriptions1,descriptions2])

In [5]:
tab_view = widgets.Tab()
tab_view.children = [FullBox, descriptions]
tab_view.set_title(0, 'Input Parameters')
tab_view.set_title(1, 'Input Descriptions')
display(tab_view)

Tab(children=(VBox(children=(VBox(children=(Label(value=' Target dimensions, nm', layout=Layout(display='flex', justify_content='center')), HBox(children=(HBox(children=(Label(value='    X:'), BoundedFloatText(value=5.0, layout=Layout(width='50px'), max=40.0, min=4.0, step=1.0)), layout=Layout(width='150px')), HBox(children=(Label(value='    Y:'), BoundedFloatText(value=5.0, layout=Layout(width='50px'), max=40.0, min=4.0, step=1.0)), layout=Layout(width='150px')), HBox(children=(Label(value='    Z:'), BoundedFloatText(value=2.0, layout=Layout(width='50px'), max=40.0, min=1.0, step=1.0)), layout=Layout(width='150px')))), HBox(children=(VBox(children=(Label(value='Ion Energy (eV):'), Label(value='# of Processors:'), Label(value='Temperature (K):'), Label(value='# of Ions to fire:'), Label(value='Output frequency:'), Label(value='$\\Delta t _{ion}$ (ps)'), Label(value='Beam Angle $\\theta$:'), Label(value='Beam Angle $\\Phi$:'), Label(value='FWHM of beam (nm):'))), VBox(children=(BoundedFloatText(value=2000.0, layout=Layout(width='80px'), max=60000.0, min=500.0, step=100.0), BoundedIntText(value=2, layout=Layout(width='80px'), max=2, min=1), BoundedFloatText(value=300.0, layout=Layout(width='80px'), max=2000.0, min=10.0, step=10.0), BoundedIntText(value=1, layout=Layout(width='80px'), min=1), BoundedIntText(value=500, layout=Layout(width='80px'), max=1000000, step=100), BoundedFloatText(value=1.0, layout=Layout(width='80px'), min=0.1, step=0.1), BoundedFloatText(value=0.0, layout=Layout(width='80px'), max=89.0, step=1.0), FloatText(value=0.0, layout=Layout(width='80px'), step=1.0), BoundedFloatText(value=2.0, layout=Layout(width='80px'), min=0.1)))))), layout=Layout(border='solid')), HBox(children=(VBox(children=(Label(value='Effective Beam Current (nA): '), Label(value='$4\\sigma$ Surface Flux (ions/cm$^2$ps): '), Label(value='Estimated Total File Size (MB)'))), VBox(children=(Text(value='160.200', layout=Layout(width='80px')), Text(value='1.1033E+13', layout=Layout(width='80px')), Text(value='13.773', layout=Layout(width='80px')))))))), VBox(children=(VBox(children=(HBox(children=(Label(value='', layout=Layout(width='80px')), Label(value='$ \\text{Descriptions (scrolling)} $', layout=Layout(flex_flow='row', height='', overflow_x='scroll', width='200px')))), HBox(children=(Label(value='Dimensions:', layout=Layout(width='80px')), Label(value='$ \\text{Simulation domain dimensions in nm. For simulation, this input is rounded to the nearest integer unit cell value.} $', layout=Layout(flex_flow='row', height='', overflow_x='scroll', width='200px')))), HBox(children=(Label(value='Ion Energy:', layout=Layout(width='80px')), Label(value='$ \\text{Kinetic Energy of the Ga ion being fired, in eV}$', layout=Layout(flex_flow='row', height='', overflow_x='scroll', width='200px')))), HBox(children=(Label(value='Processors:', layout=Layout(width='80px')), Label(value='$ \\text{The number of processors to use. Limited to 2 for local nanohub simulations. Using 2 should always be preferable, but very small domains may require 1.}$', layout=Layout(flex_flow='row', height='', overflow_x='scroll', width='200px')))), HBox(children=(Label(value='Temperature:', layout=Layout(width='80px')), Label(value='$ \\text{Silicon thermostatted temperature, applied to a 1.5 nm thick outer boundary in the x-y plane. Implemented as a Berendsen thermostat with } \\tau \\text{ = 0.1 (ps)} $', layout=Layout(flex_flow='row', height='', overflow_x='scroll', width='200px')))), HBox(children=(Label(value='Ions:', layout=Layout(width='80px')), Label(value='$ \\text{Total number of ions fired at the surface.} $', layout=Layout(flex_flow='row', height='', overflow_x='scroll', width='200px')))), HBox(children=(Label(value='Output:', layout=Layout(width='80px')), Label(value='$ \\text{Snapshot every }x \\text{ timesteps during large steps (0.2 fs), } x/10 \\text{ steps during small steps (0.02 fs)} $', layout=Layout(flex_flow='row', height='', overflow_x='scroll', width='200p

In [6]:
!touch out.log data/mdrun2.xyz data/3dmol2.xyz
#file download links
outfile = FileLink("out.log")
xyzfile = FileLink("data/mdrun2.xyz")
ovitofile = FileLink("data/OvitoTemplate.ovito")
display(outfile,xyzfile,ovitofile)

/home/nanohub/jmstout2/notebooks/FIB_MD/out.log

/home/nanohub/jmstout2/notebooks/FIB_MD/data/mdrun2.xyz

/home/nanohub/jmstout2/notebooks/FIB_MD/data/OvitoTemplate.ovito

# Function Definitions

In [7]:
##For determining how to distribute processor load in domain
#list of prime factors for some number 'n'
def prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors

#determine 3 integer factors of nc that are closest together
def primefactors3(nc):
    a=prime_factors(nc)
    z = np.ones(3,dtype=np.int8)
    zb = []

    if len(a) < 3:
        z[3-len(a):] = a
        b = range(0)
    else:
        z = z = a[len(a)-3:]
        b = range(len(a)-4,-1,-1)

    for i in b:
        indmin = np.argmin(z)
        z[indmin]=z[indmin]*a[i]    

    zb.append(z[np.argmax(z)])
    zb.append(int(np.sum(z)-z[np.argmin(z)]-z[np.argmax(z)]))
    zb.append(z[np.argmin(z)])
    return zb

#Create the input file from siga.in.template
def get_template():
    procs = primefactors3(pin.value)
    if ain.value == 0:
        atmoutin1 = -1
        atmoutin2 = -1
    else:
        atmoutin1 = int(ain.value/10)
        atmoutin2 = ain.value
        dims = [int(xdimin.value/silat), int(ydimin.value/silat), int(zdimin.value/silat)]
        
    inputs = list([str(keVin.value),str(dims[0]),str(dims[1]),str(dims[2]),
                         str(procs[0]),str(procs[1]),str(procs[2]),str(tempin.value),str(nin.value),str(atmoutin1),str(atmoutin2),
                         str(dtin.value*1e-12),str(fin.value),str(phizin.value),str(phixyin.value)])
    
    tags = list(['keV','xdim','ydim','zdim',
                 'procsx','procsy','procsz','Ttar1','nlj','atmout1','atmout2',
                 'dtion','fwhm','phiz','phixy'])

    input_dict = dict(zip(tags, inputs))
    temp_contents = open('siga.in.template').read()
    tempstr = Template(temp_contents)
    return tempstr.substitute(input_dict)


##Callback functions 
def BC_cb(change):
    BeamCurrent.value = str(1.602e-19/(dtin.value*1e-12)*1e9) #atoms/ps to nA
    
def ES_cb(change):
    if ain.value <= 0:
        EstimatedSize.value = '0'
    else:
        EstimatedSize.value = '%0.3f'%(6.5e-05*(np.floor(xdimin.value/silat) * 
                      np.floor(ydimin.value/silat) * 
                      np.floor(zdimin.value/silat))*8*
             (.1/.02e-3 /(ain.value/10) + (dtin.value-.1)/.2e-3/(ain.value))*nin.value)

def SF_cb(change):
        SurfFlux.value = '%1.4E'%(1/dtin.value/((fin.value/2.355*4*1e-7)**2*np.pi/4))
        
#button click run function
def my_start(s):
    rname = s.make_rname(pin.value)
    !rm siga.in
    with open('siga.in', 'w') as tfile:
        tfile.write(get_template())

    # run locally for now, also split output to out.log and stdout
    submit_str = '--local mpirun -np %i bin/mdrun2 | tee out.log'%(pin.value)
    s.run(rname, submit_str)
    
#button click for updating the visualization window
def updateviewer(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))
UV = widgets.Button(button_style='info',description="Update Viewer")
UV.on_click(updateviewer)


## Observers, for dynamic input/output box updates
dtin.observe(BC_cb,names = 'value')
dtin.observe(ES_cb,names = 'value')
dtin.observe(SF_cb,names = 'value')
ain.observe(ES_cb,names = 'value')
nin.observe(ES_cb,names = 'value')
xdimin.observe(ES_cb,names = 'value')
ydimin.observe(ES_cb,names = 'value')
zdimin.observe(ES_cb,names = 'value')
fin.observe(SF_cb,names = 'value')

#definition for the visualization window
p = py3Dmol.view(width=520,height=520)

In [8]:
test = Submit(start_func=my_start, cachename='SubmitTest')
test

VBox(children=(Button(button_style='success', description='Run', style=ButtonStyle(), tooltip='Run Simulation'),))

In [9]:
display(UV)

Button(button_style='info', description='Update Viewer', style=ButtonStyle())

In [10]:
#3dmol viewer
#display(widgets.Text(value = 'control+click to slide'))
#print('\n')
file = 'data/3dmol2.xyz'
xyzout = open(file,'r').read()
p.clear()
p.addModel(xyzout,'xyz')
p.setStyle({'sphere':{}})
p.setBackgroundColor('0xeeeeee')
#p.clear()
#p.zoom()
p.render()
#display(widgets.Text(value = '\n'))

